In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from action import Action
# from PDDL import PDDL_Parser
from PDDLz3 import PDDL_Parser_z3
import sys, pprint
from collections import OrderedDict
from typing import List, Tuple, Dict, Iterable
import re, copy
import itertools
import z3
from skill_classes import EffectTypePDDL, SkillPDDL
from utils import product_dict, nested_list_replace, get_atoms, get_all_objects, condition_str2objects
from scoping import scope
import time
import re

In [4]:
taxi_dom, taxi_prob = "./examples/existential-taxi/taxi-domain.pddl", "./examples/existential-taxi/prob-03.pddl"

domain, problem = taxi_dom, taxi_prob

parser = PDDL_Parser_z3()
parser.parse_domain(domain)
parser.parse_problem(problem)

skill_list = parser.get_skills()

# This below block converts all the domain's goals to z3
goal_cond = parser.get_goal_cond()

# This below block converts all the domain's initial conditions to z3
init_cond_list = parser.get_init_cond_list()

In [14]:
with open(taxi_dom, "r") as f:
    dom_s = f.read()
state_constraints_s = dom_s.split("!!!STATE_CONSTRAINT!!!")[1].split("!!!CONSTRAINT_STATE!!!")[0].strip().replace(";","")
print(state_constraints_s)

(forall (?t - taxi ?p0, ?p1 - passenger) (not (and (
     (in-taxi ?p0) (in-taxi ?p1) (not (?p0 = ?p1))
 ))))


In [12]:
dom_s.split("!!!STATE_CONSTRAINT!!!")[1]

IndexError: list index out of range

In [13]:
dom_s

';; Domain constructed by: Nishanth J. Kumar (nkumar12@cs.brown.edu)\n\n(define (domain universal_multipasstaxi)\n(:requirements :equality :typing :fluents :negative-preconditions :universal-preconditions :existential-preconditions)\n\n(:types passenger taxi - object)\n(:predicates (in-taxi ?p - passenger ?t - taxi))\n(:functions (pass-y ?p - passenger)\n            (pass-x ?p - passenger)\n            (taxi-x ?t - taxi)\n            (taxi-y ?t - taxi))\n\n\n(:action pickup\n :parameters (?p - passenger ?t - taxi)\n :precondition (and (= (pass-x ?p) (taxi-x ?t)) \n                    (= (pass-y ?p) (taxi-y ?t))\n                    (forall (?pn - passenger)\n                       (not (in-taxi ?pn ?t))\n                    )\n                )\n :effect (in-taxi ?p ?t))\n\n(:action dropoff\n :parameters (?p - passenger ?t - taxi)\n :precondition (in-taxi ?p ?t)\n :effect (not (in-taxi ?p ?t)))\n\n(:action move-north\n :parameters (?t - taxi)\n :precondition (forall (?pn - passenger)\n